In [6]:
# Load libraries
import moments
import matplotlib
import numpy as np
import pickle
from math import log

In [7]:
# Setup
dir = "/scratch/djb3ve/connor/"
pickle_file = dir + "run_moments/daphnia.filtered.chr.busco_data_dict.pickle"
#pickle_file = dir + "run_moments/daphnia.filt.mlg.genome.11.18.22_data_dict.pickle"
output_file = dir + "sfs_statistics.txt"
csv_dir = dir + "sfs_csvs/"
sfs_dir = dir + "sfss/"
pop_ids = ["Daphnia.pulex.NorthAmerica", "Daphnia.pulex.Europe"]
prior_estimated_params = [7e5, 2e5, 1e7 / 2, 1e-8 * 2]
split_mig_params = [6.358476, 1.032427, 9.989897, 0.007727318]
split_no_mig_params = [1.065988, 0.2213014, 0.8256301, 0]
#split_mig_params = [6.758185, 1.1126711, 9.9724427, 0.008836859]
#split_no_mig_params = [1.127693, 0.2289919, 0.7780228, 0]

In [3]:
def save_sfs_as_csv_and_npy(sfs, sfs_name):
    # Save SFS as CSV
    np.savetxt(csv_dir + sfs_name + ".csv", sfs, delimiter=",")
    # Save SFS as numpy binary
    with open(sfs_dir + sfs_name + ".npy", "wb") as fout:
        np.save(fout, sfs.data)

def get_AIC(sfs, sfs_empirical, k):
    # k is the number of model parameters.
    ll = moments.Inference.ll_multinom(sfs, sfs_empirical)
    AIC = k * 2 - 2 * ll
    return AIC

def get_BIC(sfs, sfs_empirical, k, sfs_size):
    # k is the number of model parameters.
    # The number of elements in a square folded SFS with the absent allele corner
    # masked is the ith triangular number minus 1, where i is the sample size.
    n = sfs_size * (sfs_size + 1) / 2 - 1
    ll = moments.Inference.ll_multinom(sfs, sfs_empirical)
    BIC = k * log(n) - 2 * ll
    return BIC

def get_shared_allele_prop(sfs, maf_threshold=0.01):
    # Iterate through sfs, summing element values, but skipping entries corresponding
    # to alleles that are not sufficiently shared according to "maf_threshold"
    shared_allele_count = 0
    coordinate_thresholds = np.array([(len - 1) * maf_threshold + 1 for len in sfs.shape]).astype(int)

    # Create NumPy iterator object
    it = np.nditer(sfs, flags=['multi_index'])
    for i in it:
        shared_allele = True
        # Check for whether element should be skipped because it fails to cross
        # MAF threshold
        for j, coord in enumerate(it.multi_index):
            if coord < coordinate_thresholds[j]:
                shared_allele = False
                break

        # If it crosses the MAF threshold to be considered "shared", add it to the counter
        if shared_allele:
            shared_allele_count += i

    shared_allele_prop = shared_allele_count / np.sum(sfs)
    return shared_allele_prop

def write_output(fout, outputs):
    for output in outputs:
        fout.write(str(output) + "\t")
    fout.write("\n")

In [8]:
with open(pickle_file, "rb") as f:
    data_dict = pickle.load(f)

In [8]:
ns = [20, 20]

In [10]:
sfs_empirical = moments.Spectrum.from_data_dict(data_dict, pop_ids=pop_ids,
                                                  projections=ns,
                                                  polarized=False)

In [ ]:
sfs_from_ests = moments.Demographics2D.split_mig(prior_estimated_params, ns,
                                                   pop_ids=pop_ids).fold()
sfs_split_mig_model = moments.Demographics2D.split_mig(split_mig_params, ns,
                                                              pop_ids=pop_ids).fold()
sfs_split_no_mig_model = moments.Demographics2D.split_mig(split_no_mig_params, ns,
                                                                 pop_ids=pop_ids).fold()

In [23]:
moments.Inference.ll_multinom(sfs_from_ests, sfs_empirical / np.sum(sfs_empirical))

-3.774635228397063

In [24]:
moments.Inference.ll_multinom(sfs_split_mig_model, sfs_empirical / np.sum(sfs_empirical))

-3.489374807789013

In [25]:
moments.Inference.ll_multinom(sfs_split_no_mig_model, sfs_empirical / np.sum(sfs_empirical))

-3.517175603071643

In [13]:
pickle_file = dir + "run_moments/daphnia.filt.mlg.genome.11.18.22_data_dict.pickle"
split_mig_params = [6.758185, 1.1126711, 9.9724427, 0.008836859]
split_no_mig_params = [1.127693, 0.2289919, 0.7780228, 0]
with open(pickle_file, "rb") as f:
    data_dict_filt = pickle.load(f)
sfs_empirical_filt = moments.Spectrum.from_data_dict(data_dict_filt, pop_ids=pop_ids,
                                                  projections=ns,
                                                  polarized=False)

In [14]:
sfs_from_ests_filt = moments.Demographics2D.split_mig(prior_estimated_params, ns,
                                                   pop_ids=pop_ids).fold()
sfs_split_mig_model_filt = moments.Demographics2D.split_mig(split_mig_params, ns,
                                                              pop_ids=pop_ids).fold()
sfs_split_no_mig_model_filt = moments.Demographics2D.split_mig(split_no_mig_params, ns,
                                                                 pop_ids=pop_ids).fold()

In [16]:
moments.Inference.ll_multinom(sfs_from_ests_filt, sfs_empirical_filt / np.sum(sfs_empirical_filt))

-3.8305639089964485

In [18]:
moments.Inference.ll_multinom(sfs_split_mig_model_filt, sfs_empirical_filt / np.sum(sfs_empirical_filt))

-3.513468319173573

In [19]:
moments.Inference.ll_multinom(sfs_split_no_mig_model_filt, sfs_empirical_filt / np.sum(sfs_empirical_filt))

-3.5417243695695

In [9]:
ns = [100, 100]

In [10]:
sfs_empirical = moments.Spectrum.from_data_dict(data_dict, pop_ids=pop_ids,
                                                  projections=ns,
                                                  polarized=False)

In [11]:
sfs_from_ests = moments.Demographics2D.split_mig(prior_estimated_params, ns,
                                                   pop_ids=pop_ids).fold()
sfs_split_mig_model = moments.Demographics2D.split_mig(split_mig_params, ns,
                                                              pop_ids=pop_ids).fold()
sfs_split_no_mig_model = moments.Demographics2D.split_mig(split_no_mig_params, ns,
                                                                 pop_ids=pop_ids).fold()

In [12]:
moments.Inference.ll_multinom(sfs_from_ests, sfs_empirical / np.sum(sfs_empirical))

-4.758038564030496

In [13]:
moments.Inference.ll_multinom(sfs_split_mig_model, sfs_empirical / np.sum(sfs_empirical))

-4.467354420192853

In [14]:
moments.Inference.ll_multinom(sfs_split_no_mig_model, sfs_empirical / np.sum(sfs_empirical))

-4.534079265957455